In [ ]:
import pandas as pd
from elasticsearch7 import Elasticsearch
import csv
import os
import dotenv

dotenv.load_dotenv()

True

In [2]:
columns = ['id', 'dataNotificacao', 'sexo', 'racaCor', 'estado','estadoIBGE', 'municipio', 'municipioIBGE', 
           'estadoNotificacao', 'estadoNotificacaoIBGE', 'municipioNotificacao', 'municipioNotificacaoIBGE', 
           'sintomas', 'outrosSintomas', 'dataInicioSintomas', 'dataEncerramento', 'evolucaoCaso', 'classificacaoFinal',
           'resultadoTeste', 'codigoTriagemPopulacaoEspecifica', 'resultadoTesteSorologicoIgG', 'outrasCondicoes', 
           'idade', 'loteSegundaReforcoDose', 'profissionalSaude', 'tipoTeste', 'resultadoTesteSorologicoIgM', 
           'resultadoTesteSorologicoTotais', 'qualAntiviral', 'codigoContemComunidadeTradicional', '@version', 
           'dataTesteSorologico', 'estrangeiro', 'idCollection', 'codigoDosesVacina', 'codigoLocalRealizacaoTestagem',  
           'laboratorioSegundaReforcoDose', 'dataSegundaDose', 'codigoEstrategiaCovid', 'dataSegundaReforcoDose', 
           'codigoRecebeuAntiviral', 'outroTriagemPopulacaoEspecifica', '@timestamp',  'outroBuscaAtivaAssintomatico', 
           'codigoBuscaAtivaAssintomatico', 'recebeuAntiviral', 'codigoRecebeuVacina', 'dataReforcoDose', 
           'tipoTesteSorologico', 'dataInicioTratamento', 'cbo', 'registroAtual', 'dataPrimeiraDose', 'condicoes', 
           'dataTeste', 'outroAntiviral', 'estadoTeste', 'codigoQualAntiviral', 'outroLocalRealizacaoTestagem']

index = "desc-esus-notifica-estado-*"
output_file = "sindrome_gripal.csv"

print(f"Index: {index}")

user = os.getenv("ES_USER")
pwd = os.getenv("ES_PASSWORD")
protocol = 'https'
url = f"{protocol}://{user}:{pwd}@elasticsearch-saps.saude.gov.br/"
es = Elasticsearch([url], send_get_body_as='POST', timeout=2000)

query_body = {
    "query": {
        "bool": {
            "must": {"match_all": {}},
            "filter": {
                "range": {
                    "dataInicioSintomas": {
                        "gte": "2024-01-01T00:00:00", 
                        "lte": "now"
                    }
                }
            }
        }
    }
}

# Contar total de documentos
total_documents = es.count(index=index, body=query_body)['count']
print(f"Total de documentos no índice: {total_documents}")

# Scroll params
scroll_time = '1m'
scroll_size = 10000

# Primeira busca
response = es.search(
    index=index,
    scroll=scroll_time,
    size=scroll_size,
    body=query_body
)

scroll_id = response['_scroll_id']
retrieved_documents = 0

# Abre o arquivo CSV e escreve os dados aos poucos
with open(output_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=columns)
    writer.writeheader()

    while True:
        hits = response['hits']['hits']
        if not hits:
            break

        for hit in hits:
            source = hit['_source']
            row = {col: source.get(col, "") for col in columns}
            writer.writerow(row)

        retrieved_documents += len(hits)
        print(f"Progresso: {retrieved_documents}/{total_documents} ({(retrieved_documents / total_documents) * 100:.2f}%)")

        # Próximo scroll
        response = es.scroll(scroll_id=scroll_id, scroll=scroll_time)
        scroll_id = response['_scroll_id']

# Liberar scroll
es.clear_scroll(scroll_id=scroll_id)
print(f"Download completo. Total de documentos salvos: {retrieved_documents}")
print(f"Arquivo salvo em: {output_file}")


Index: desc-esus-notifica-estado-*
Total de documentos no índice: 5492179
Progresso: 10000/5492179 (0.18%)
Progresso: 20000/5492179 (0.36%)
Progresso: 30000/5492179 (0.55%)
Progresso: 40000/5492179 (0.73%)
Progresso: 50000/5492179 (0.91%)
Progresso: 60000/5492179 (1.09%)
Progresso: 70000/5492179 (1.27%)
Progresso: 80000/5492179 (1.46%)
Progresso: 90000/5492179 (1.64%)
Progresso: 100000/5492179 (1.82%)
Progresso: 110000/5492179 (2.00%)
Progresso: 120000/5492179 (2.18%)
Progresso: 130000/5492179 (2.37%)
Progresso: 140000/5492179 (2.55%)
Progresso: 150000/5492179 (2.73%)
Progresso: 160000/5492179 (2.91%)
Progresso: 170000/5492179 (3.10%)
Progresso: 180000/5492179 (3.28%)
Progresso: 190000/5492179 (3.46%)
Progresso: 200000/5492179 (3.64%)
Progresso: 210000/5492179 (3.82%)
Progresso: 220000/5492179 (4.01%)
Progresso: 230000/5492179 (4.19%)
Progresso: 240000/5492179 (4.37%)
Progresso: 250000/5492179 (4.55%)
Progresso: 260000/5492179 (4.73%)
Progresso: 270000/5492179 (4.92%)
Progresso: 280000

KeyboardInterrupt: 